In [1]:
import time
import collections
import random
import numpy as np
import gym
import keras.layers
import keras.models
from keras import backend as K

Using TensorFlow backend.


In [2]:
def env_create():
    env = gym.make('LunarLander-v2')
    return env
env = env_create()
env_max_possible_score = 200
print(env.reset())
print(env.observation_space)
print(env.action_space)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-0.00433264  0.94127761 -0.43886337  0.04427049  0.00502723  0.0994091
  0.          0.        ]
Box(8,)
Discrete(4)


In [3]:
model_sample = collections.namedtuple('model_sample',
    ['state', 'action', 'reward'])

In [4]:
class model:
    
    def __init__(self, input_n, output_n, beta=0.0001):
        
        self.__input_n = input_n
        self.__output_n = output_n
        self.__l_shared = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv1D(64, 3, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.LSTM(128, return_sequences=False),
            keras.layers.Dense(128, kernel_initializer='he_uniform'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu')]
        self.__l_policy = [
            keras.layers.Dense(128, kernel_initializer='he_uniform'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dense(128, kernel_initializer='he_uniform'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dense(self.__output_n, activation='softmax')]
        self.__l_value = [
            keras.layers.Dense(128, kernel_initializer='he_uniform'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dense(128, kernel_initializer='he_uniform'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dense(1)]
        def apply_layers(x, layers):
            last_layer = x
            for l in layers:
                last_layer = l(last_layer)
            return last_layer
        
        m = m_input = keras.layers.Input((None, self.__input_n,))
        m = apply_layers(m, self.__l_shared)
        m = apply_layers(m, self.__l_policy)
        self.__m_policy = keras.models.Model([m_input], [m])
        
        m = m_input = keras.layers.Input((None, self.__input_n,))
        m = apply_layers(m, self.__l_shared)
        m = apply_layers(m, self.__l_value)
        self.__m_value = keras.models.Model([m_input], [m])
        
        m = m_input = keras.layers.Input((None, self.__input_n,))
        m_shared = apply_layers(m, self.__l_shared)
        m_policy = apply_layers(m_shared, self.__l_policy)
        m_value = apply_layers(m_shared, self.__l_value)
        self.__m_value_policy = keras.models.Model([m_input], [m_value, m_policy])
        
        m = m_input = keras.layers.Input((None, self.__input_n,))
        m_value, m_policy = self.__m_value_policy(m)
        m = keras.layers.Concatenate()([m_value, m_policy])
        self.__m_train = keras.models.Model([m_input], [m])
        self.__m_train.compile('nadam',
            lambda y_true, y_pred: model.__loss(y_true, y_pred, beta))
        
        self.__m_policy.summary()
        self.__m_value.summary()
        self.__m_train.summary()
    
    @staticmethod
    def __loss(y_true, y_pred, beta):
        r, action_onehot = y_true[:,:1], y_true[:,1:]
        value, policy = y_pred[:,:1], y_pred[:,1:]
        advantage = r - value
        log_policy = K.log(policy + K.epsilon())
        log_choosen_action_prob = K.sum(action_onehot * log_policy, axis=-1, keepdims=True)
        action_loss = -K.mean(log_choosen_action_prob * advantage)
        value_loss = 0.5 * K.mean(K.square(advantage))
        entropy = K.mean(-K.sum(policy * log_policy, axis=-1, keepdims=True))
        return action_loss + value_loss - beta * entropy
    
    def train(self, samples, epochs=1, verbose=False):
        self.__m_train.fit(
            x=np.array([s.state for s in samples], dtype=np.float32),
            y=np.hstack([
                np.reshape(np.array([s.reward for s in samples], dtype=np.float32), (-1, 1)),
                keras.utils.to_categorical([s.action for s in samples], num_classes=self.__output_n)]),
            batch_size=64,
            epochs=epochs,
            verbose=verbose)
    
    def evalute_value(self, state, verbose=False):
        v = self.__m_value.predict(
            np.array([state], dtype=np.float32))[0,0]
        if verbose:
            print(v)
        return v
    
    def get_action_prob(self, state, verbose=False):
        action_prob = self.__m_policy.predict(
            np.array([state], dtype=np.float32))[0]
        if verbose:
            print(action_prob)
        return action_prob

In [5]:
def play(env, m, gamma=0.98, max_steps=1000, n_prev_states=8, epsilon=0., verbose=False):
    state_0 = env.reset()
    state_null = np.zeros_like(state_0)
    state_queue = []
    episode = []
    samples = []
    action_probs = []
    gamelen = 0
    gamelen_max = 0
    def get_prev_states(episode, idx):
        states = [e.state for e in episode[max(0,(idx-n_prev_states)+1):idx+1]]
        states = [state_null]*(max(0,n_prev_states-len(states))) + states
        return states
    def add_to_samples(episode, done):
        if done:
            discounted_reward = 0.
        else:
            discounted_reward = m.evalute_value(get_prev_states(episode, len(episode)-1))
        episode[-1] = model_sample(
                get_prev_states(episode, len(episode)-1),
                episode[-1].action,
                discounted_reward)
        for i in reversed(range(len(episode)-1)):
            discounted_reward = episode[i].reward + \
                gamma * discounted_reward
            episode[i] = model_sample(
                get_prev_states(episode, i),
                episode[i].action,
                discounted_reward)
        samples.extend(episode)
    for i in range(max_steps):
        state_queue.append(state_0)
        if len(state_queue) > n_prev_states:
            state_queue.pop(0)
        state_queue_padded = \
            [state_null]*(max(0,n_prev_states-len(state_queue))) + state_queue
        action_prob = m.get_action_prob(state_queue_padded)
        action_probs.append(action_prob)
        action_prob = epsilon/action_prob.shape[-1] + (1-epsilon)*action_prob
        action = int(np.random.choice(
            list(range(action_prob.shape[-1])),
            p=action_prob))
        state_1, reward, done, _ = env.step(action)
        episode.append(model_sample(state_0, action, reward))
        state_0 = state_1
        gamelen += 1
        if done:
            add_to_samples(episode, True)
            episode = []
            state_0 = env.reset()
            state_null = np.zeros_like(state_0)
            state_queue = []
            gamelen_max = max(gamelen_max, gamelen)
            gamelen = 0
    if episode:
        add_to_samples(episode, False)
        gamelen_max = max(gamelen_max, gamelen)
        gamelen = 0
    if verbose:
        print('std[action_prob]', np.mean(np.std(action_probs, ddof=1, axis=0)))
        print('max game len', gamelen_max)
    return samples

In [6]:
K.clear_session()
m = model(8, 4, beta=0.0001)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 8)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 8)           32        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          1600      
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 64)          256       
_________________________________________________________________
activation_1 (Activation)    (None, None, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
__________

In [7]:
def test(m, render=False, n_prev_states=8):
    global env
    state = env.reset()
    state_null = np.zeros_like(state)
    state_queue = []
    rewards = 0.
    while True:
        state_queue.append(state)
        if len(state_queue) > n_prev_states:
            state_queue.pop(0)
        state_queue_padded = \
            [state_null]*(max(0,n_prev_states-len(state_queue))) + state_queue 
        action_prob = m.get_action_prob(state_queue)
        action = np.argmax(action_prob)
        state, reward, done, _ = env.step(action)
        rewards += reward
        if render:
            env.render()
        if done:
            break
        if render:
            time.sleep(1/60)
    if render:
        env.close()
        env = env_create()
    return rewards

In [8]:
replays = []
for i in range(100):
    samples = play(
        env, m, max_steps=3000, n_prev_states=64, epsilon=0.25, verbose=True)
    replays.extend(samples)
    m.train(replays, epochs=15, verbose=True)
    if len(replays) > 10000:
        random.shuffle(replays)
        replays = replays[:10000]
    print('epoch {} completed'.format(i))
    test_result = [test(m, n_prev_states=64, render=False) for _ in range(10)]
    print('test result',
          'mean', np.mean(test_result),
          'std', np.std(test_result, ddof=1))
    test(m, n_prev_states=64, render=True)
    if np.mean(test_result) == env_max_possible_score and \
        np.std(test_result, ddof=1) == 0:
        print('the network always gets full score, early exit')
        break

std[action_prob] 0.0077664307
max game len 155
Epoch 1/15
3000/3000 [==============================] - 7s 2ms/step - loss: 1361.2545
Epoch 2/15
3000/3000 [==============================] - 4s 1ms/step - loss: 793.6019
Epoch 3/15
3000/3000 [==============================] - 4s 1ms/step - loss: 456.4656
Epoch 4/15
3000/3000 [==============================] - 4s 1ms/step - loss: 253.8515
Epoch 5/15
3000/3000 [==============================] - 4s 1ms/step - loss: 155.8841
Epoch 6/15
3000/3000 [==============================] - 4s 1ms/step - loss: 119.8083
Epoch 7/15
3000/3000 [==============================] - 4s 1ms/step - loss: 98.4216
Epoch 8/15
3000/3000 [==============================] - 4s 1ms/step - loss: 54.3075
Epoch 9/15
3000/3000 [==============================] - 4s 1ms/step - loss: 41.3258
Epoch 10/15
3000/3000 [==============================] - 4s 1ms/step - loss: 63.2663
Epoch 11/15
3000/3000 [==============================] - 4s 1ms/step - loss: 47.5470
Epoch 12/15
3000/300

13000/13000 [==============================] - 16s 1ms/step - loss: -9.4743
Epoch 8/15
13000/13000 [==============================] - 16s 1ms/step - loss: -1.4613
Epoch 9/15
13000/13000 [==============================] - 16s 1ms/step - loss: -11.3922
Epoch 10/15
13000/13000 [==============================] - 16s 1ms/step - loss: 0.3853
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: -17.2182
Epoch 12/15
13000/13000 [==============================] - 16s 1ms/step - loss: 0.4743
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: -8.1846
Epoch 14/15
13000/13000 [==============================] - 16s 1ms/step - loss: -11.1607
Epoch 15/15
13000/13000 [==============================] - 16s 1ms/step - loss: -20.7182
epoch 5 completed
test result mean -240.04897504729956 std 86.00053851192193
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
std[action_prob] 0.4220218
max game len 153


13000/13000 [==============================] - 16s 1ms/step - loss: 49.7604
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: 67.2117
Epoch 14/15
13000/13000 [==============================] - 16s 1ms/step - loss: 83.1351
Epoch 15/15
13000/13000 [==============================] - 16s 1ms/step - loss: 53.6271
epoch 10 completed
test result mean -220.1989415554402 std 152.31415960087236
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
std[action_prob] 0.41296253
max game len 147
Epoch 1/15
13000/13000 [==============================] - 16s 1ms/step - loss: 207.8128
Epoch 2/15
13000/13000 [==============================] - 16s 1ms/step - loss: 134.4203
Epoch 3/15
13000/13000 [==============================] - 16s 1ms/step - loss: 108.4349
Epoch 4/15
13000/13000 [==============================] - 16s 1ms/step - loss: 112.8971
Epoch 5/15
13000/13000 [==============================] - 16s 1ms/step - loss: 88.5838

std[action_prob] 0.4172333
max game len 121
Epoch 1/15
13000/13000 [==============================] - 16s 1ms/step - loss: 113.3742
Epoch 2/15
13000/13000 [==============================] - 16s 1ms/step - loss: 71.4524
Epoch 3/15
13000/13000 [==============================] - 16s 1ms/step - loss: 51.5559
Epoch 4/15
13000/13000 [==============================] - 16s 1ms/step - loss: 51.6252
Epoch 5/15
13000/13000 [==============================] - 16s 1ms/step - loss: 50.9449
Epoch 6/15
13000/13000 [==============================] - 16s 1ms/step - loss: 40.6358
Epoch 7/15
13000/13000 [==============================] - 16s 1ms/step - loss: 41.6561
Epoch 8/15
13000/13000 [==============================] - 16s 1ms/step - loss: 47.1673
Epoch 9/15
13000/13000 [==============================] - 16s 1ms/step - loss: 41.7540
Epoch 10/15
13000/13000 [==============================] - 16s 1ms/step - loss: 25.5210
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: 15.2

13000/13000 [==============================] - 16s 1ms/step - loss: 45.7555
Epoch 7/15
13000/13000 [==============================] - 16s 1ms/step - loss: 27.9540
Epoch 8/15
13000/13000 [==============================] - 16s 1ms/step - loss: 32.2208
Epoch 9/15
13000/13000 [==============================] - 16s 1ms/step - loss: 25.8922
Epoch 10/15
13000/13000 [==============================] - 16s 1ms/step - loss: 19.2256
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: 18.4854
Epoch 12/15
13000/13000 [==============================] - 16s 1ms/step - loss: 48.6186
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: 40.8577
Epoch 14/15
13000/13000 [==============================] - 16s 1ms/step - loss: 40.6674
Epoch 15/15
13000/13000 [==============================] - 16s 1ms/step - loss: 20.3456
epoch 21 completed
test result mean -216.20564628454636 std 92.25862395176112
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.

...

13000/13000 [==============================] - 16s 1ms/step - loss: -10.4588
Epoch 6/15
13000/13000 [==============================] - 16s 1ms/step - loss: -11.0263
Epoch 7/15
13000/13000 [==============================] - 16s 1ms/step - loss: -17.4299
Epoch 8/15
13000/13000 [==============================] - 16s 1ms/step - loss: -15.1058
Epoch 9/15
13000/13000 [==============================] - 16s 1ms/step - loss: -20.4929
Epoch 10/15
13000/13000 [==============================] - 16s 1ms/step - loss: -19.3904
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: -25.4326
Epoch 12/15
13000/13000 [==============================] - 16s 1ms/step - loss: -22.7303
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: -31.3282
Epoch 14/15
13000/13000 [==============================] - 16s 1ms/step - loss: -32.2988
Epoch 15/15
13000/13000 [==============================] - 16s 1ms/step - loss: -32.6801
epoch 74 completed
test result mean -

13000/13000 [==============================] - 16s 1ms/step - loss: -7.0881
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: -14.3759
Epoch 12/15
13000/13000 [==============================] - 16s 1ms/step - loss: -16.3721
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: -23.9948
Epoch 14/15
13000/13000 [==============================] - 16s 1ms/step - loss: -10.8046
Epoch 15/15
13000/13000 [==============================] - 16s 1ms/step - loss: -15.1573
epoch 79 completed
test result mean -185.40598763331712 std 86.1634720338438
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
std[action_prob] 0.41088325
max game len 229
Epoch 1/15
13000/13000 [==============================] - 16s 1ms/step - loss: 91.2695
Epoch 2/15
13000/13000 [==============================] - 16s 1ms/step - loss: 54.4024
Epoch 3/15
13000/13000 [==============================] - 16s 1ms/step - loss: 35.30

13000/13000 [==============================] - 16s 1ms/step - loss: 6.1649
epoch 84 completed
test result mean -183.3154142271406 std 86.46240236951012
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
std[action_prob] 0.4177508
max game len 149
Epoch 1/15
13000/13000 [==============================] - 16s 1ms/step - loss: 80.6515
Epoch 2/15
13000/13000 [==============================] - 16s 1ms/step - loss: 74.3046
Epoch 3/15
13000/13000 [==============================] - 16s 1ms/step - loss: 48.0179
Epoch 4/15
13000/13000 [==============================] - 16s 1ms/step - loss: 41.4967
Epoch 5/15
13000/13000 [==============================] - 16s 1ms/step - loss: 29.8110
Epoch 6/15
13000/13000 [==============================] - 16s 1ms/step - loss: 22.2731
Epoch 7/15
13000/13000 [==============================] - 16s 1ms/step - loss: 16.8538
Epoch 8/15
13000/13000 [==============================] - 16s 1ms/step - loss: 16.5731
Epoch 9/1

13000/13000 [==============================] - 16s 1ms/step - loss: 31.8522
Epoch 4/15
13000/13000 [==============================] - 16s 1ms/step - loss: 26.0276
Epoch 5/15
13000/13000 [==============================] - 16s 1ms/step - loss: 30.1251
Epoch 6/15
13000/13000 [==============================] - 16s 1ms/step - loss: 19.1399
Epoch 7/15
13000/13000 [==============================] - 16s 1ms/step - loss: 14.8117
Epoch 8/15
13000/13000 [==============================] - 16s 1ms/step - loss: 10.4468
Epoch 9/15
13000/13000 [==============================] - 16s 1ms/step - loss: 8.6270
Epoch 10/15
13000/13000 [==============================] - 16s 1ms/step - loss: -2.1500
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: 8.2481
Epoch 12/15
13000/13000 [==============================] - 16s 1ms/step - loss: 7.4180
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: 8.5163
Epoch 14/15
13000/13000 [=============================

13000/13000 [==============================] - 16s 1ms/step - loss: 26.8847
Epoch 9/15
13000/13000 [==============================] - 16s 1ms/step - loss: 26.2792
Epoch 10/15
13000/13000 [==============================] - 16s 1ms/step - loss: 24.9440
Epoch 11/15
13000/13000 [==============================] - 16s 1ms/step - loss: 26.6249
Epoch 12/15
13000/13000 [==============================] - 16s 1ms/step - loss: 19.2444
Epoch 13/15
13000/13000 [==============================] - 16s 1ms/step - loss: 22.9047
Epoch 14/15
13000/13000 [==============================] - 16s 1ms/step - loss: 16.6386
Epoch 15/15
13000/13000 [==============================] - 16s 1ms/step - loss: 16.0761
epoch 95 completed
test result mean -114.36654230372785 std 59.65701713024552
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
std[action_prob] 0.4141173
max game len 219
Epoch 1/15
13000/13000 [==============================] - 16s 1ms/step - loss: 89.9465
E

In [9]:
test(m, n_prev_states=64, render=True)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


-87.17389892619222